In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import censusdata
import re
import geopandas



In [2]:
#Define Funtions

#Funcion For NonDigit Removing Characters from Row
def remove_chars(s):
    clean = int(re.sub('[^0-9]+', '', str(s))[-11:])
    return clean

#Function to get census data according to requirements
def getcensustables(table):
    tablename = 'data_' + table
    print(tablename)
    
    # Set requirements to pull from Census API
    
    variablestopull = allvariablescsv.loc[allvariablescsv['Group']==table]['Name'].tolist()
    #print(variablestopull)
    
    #variablestopull = variables['Name'].tolist()

    #variablestopull = variables['Name'].tolist()
    listofcounties = ['121', '089']
    yearstopull = range(2012,2018,1)

    #Pull Census Data according to requirement above
    atlantadata = pd.DataFrame()

    for year in yearstopull:
        for county in listofcounties:

            newdata = censusdata.download('acs5', year, censusdata.censusgeo([('state', '13'), ('county', county), ('tract', '*')]),variablestopull)
            type(newdata)
            newdata['YEAR'] = year
            atlantadata = atlantadata.append(newdata)

    #Rename Column Headings from Code to Text
    for variable in variablestopull:
        #print(variable)
        renamevar1 = allvariablescsv.loc[allvariablescsv['Name']==variable]['Label'].iloc[0]
        renamevar2 = allvariablescsv.loc[allvariablescsv['Name']==variable]['Concept'].iloc[0]
        renamevar = renamevar2 + '_'+ renamevar1
        #print(renamevar)
        atlantadata = atlantadata.rename(columns={variable: renamevar})

    #Transform Index to create GeoID


    #Create GeoID Column
    atlantadata['Name2'] = atlantadata.index
    atlantadata['GEOID'] = atlantadata['Name2'].apply(remove_chars)
    atlantadata.drop(columns=['Name2'], inplace=True)
    
    #Only Keep Atlanta Census Tracts(Geocodes)
    finaldata = atlantageocodes.merge(atlantadata, how='left', on='GEOID')
    
    finaldata = finaldata.dropna(axis=0, how='any')
    
    return finaldata

In [3]:
#Code Begins here

#Tables to Load
tablestoloadcsv = pd.read_csv('data/tablestopull.csv')
tablestoload = tablestoloadcsv['Tables'].tolist()

#Allvariables Table
allvariablescsv = pd.read_csv('data/allvariables.csv')

#Atlanta Geo Codes Table
atlantageocodes = pd.read_csv('data/atlantageocodes.csv')

#Create dictionary with each cleaned table as an entry
alltables = dict()  

for table in tablestoload:
    data = getcensustables(table)
    alltables[table] = data
    
    data.to_csv('data/' + table + '.csv')

FileNotFoundError: [Errno 2] File b'data/tablestopull.csv' does not exist: b'data/tablestopull.csv'

In [ ]:
#Example Usage of Occupancy Over Time

occupancytable = alltables['B25002']

occupancytable = occupancytable.groupby('YEAR').sum()
occupancytable['Percent_Occupied'] = (occupancytable['OCCUPANCY STATUS_Estimate!!Total!!Occupied'] / occupancytable['OCCUPANCY STATUS_Estimate!!Total'])*100 

#print(occupancytable.head())

occupancytable = occupancytable.reset_index()


plt.plot(occupancytable['YEAR'], occupancytable['Percent_Occupied'])
plt.title("Occupancy Over Time", fontsize=18)
plt.xlabel("Years", fontsize=18)
plt.ylabel("Occupancy %", fontsize=18)


In [ ]:
occupancytable = alltables['B25002']

occupancytable['Percent_Occupied'] = (occupancytable['OCCUPANCY STATUS_Estimate!!Total!!Occupied'] / occupancytable['OCCUPANCY STATUS_Estimate!!Total'])*100 


geotable = geopandas.read_file('tractdata/tl_2018_13_tract.shp')
#geotable = geotable.astype('int64').dtypes
geotable[['GEOID']] = geotable[['GEOID']].astype('int64')

geotable2 = atlantageocodes.merge(geotable, how='left', on='GEOID')

geotable2 = geotable.set_index('GEOID').join(occupancytable.set_index('GEOID'))

geotable2 = geotable2.dropna(axis=0, how='any')

geotable2.head()

# set a variable that will call whatever column we want to visualise on the map
variable = 'Percent_Occupied'
# set the range for the choropleth
vmin, vmax = 120, 220
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(10, 10))

# create map
geotable2.plot(column=variable, cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')

plt.title("Occupancy Rate By Area", fontsize=18)

In [ ]:
occupancytable = alltables['B25002']

occupancytable['Percent_Occupied'] = (occupancytable['OCCUPANCY STATUS_Estimate!!Total!!Occupied'] / occupancytable['OCCUPANCY STATUS_Estimate!!Total'])*100 


geotable = geopandas.read_file('tractdata/tl_2018_13_tract.shp')
#geotable = geotable.astype('int64').dtypes
geotable[['GEOID']] = geotable[['GEOID']].astype('int64')

geotable2 = atlantageocodes.merge(geotable, how='left', on='GEOID')

geotable2 = geotable.set_index('GEOID').join(occupancytable.set_index('GEOID'))

geotable2 = geotable2.dropna(axis=0, how='any')

geotable2.head()

# set a variable that will call whatever column we want to visualise on the map
variable = 'Percent_Occupied'
# set the range for the choropleth
vmin, vmax = 120, 220
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(10, 10))

# create map
geotable2.plot(column=variable, cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')

plt.title("Occupancy Rate By Area", fontsize=18)